LSD profiles

## 0. Import packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Toy model

In [4]:
Z = np.array([0, 1, 2, 3, 2, 1, 0])
lines = [1, 3, 8]
M = np.zeros(12)
M[lines]=1

Create your own function to calculate a convolution

In [5]:
def my_convol(M, Z):
    return()